In [305]:
import os
import sys
module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [188]:
import math
import numpy as np
from phase1 import load_index, preprocess, read_json, get_documents, get_title_url

In [27]:
def calculate_tf(term, doc_id):
    freq = term.freq_in_doc[doc_id]
    if freq > 0:
        return 1 + math.log10(term.freq_in_doc[doc_id])
    return 0

In [62]:
def calculate_idf(term, collection_size):
    return math.log10(collection_size / len(term.freq_in_doc))

In [131]:
def calculate_tf_query(term, tokens):
    freq = 0
    for token in tokens:
        if term == token:
            freq += 1
    if freq > 0:
        return 1 + math.log10(freq)
    return 0

In [63]:
def get_tf_idf(term, doc_id, collection_size):
    tf = calculate_tf(term, doc_id)
    idf = calculate_idf(term, collection_size)
    return tf * idf

In [60]:
def compute_weights(dictionary, collection_size):
    term_weight = {}
    for term in dictionary:
        for doc_id in dictionary[term].get_docs():
            weight = get_tf_idf(dictionary[term], doc_id, collection_size)
            if term not in term_weight:
                term_weight[term] = {}
            term_weight[term][doc_id] = weight
    return term_weight

In [106]:
def get_docs_norm(weight_dict, collection_size):
    lengths = np.zeros(collection_size)
    for term in weight_dict:
        for doc_id, weight in weight_dict[term].items():
            lengths[int(doc_id)] += weight ** 2
    return np.sqrt(lengths)

In [87]:
def create_champion_list(term_weights, r):
    champs = {}
    for term in term_weights:
        champs[term] = dict(sorted(term_weights[term].items(), key=lambda item: item[1], reverse=True)[:r])
    return champs

In [259]:
def cosine_similarity(query_tokens, weights_dict, dictionary, collection_size, norms):
    scores = {}
    for term in query_tokens:
        if term in weights_dict:
            token_weight = calculate_tf_query(term, query_tokens) * calculate_idf(dictionary[term], collection_size)
            for doc_id, weight in weights_dict[term].items():
                if doc_id not in scores:
                    scores[doc_id] = 0
                scores[doc_id] += token_weight * weight 
                
    for doc_id in scores:
        scores[doc_id] /= norms[int(doc_id)]    
    
    return scores

In [204]:
def print_result(data_json, socres, k):
    sorted_scores = dict(sorted(socres.items(), key=lambda item: item[1],reverse=True)[:k])
    for doc_id in sorted_scores:
        title, url = get_title_url(data_json, doc_id)
        print('title: ', title, '\nurl: ', url)

In [43]:
inverted_index = load_index('index_without_stop_words.json')

In [50]:
json_data = read_json('IR_data_news_12k.json')
documents = get_documents(json_data)

In [94]:
N = len(documents)

In [64]:
term_weights_dict = compute_weights(inverted_index, N)

In [88]:
champion_list = create_champion_list(term_weights_dict, 50)

In [181]:
docs_norms = get_docs_norm(term_weights_dict, N)

In [205]:
query = 'فوتبال'
query_tokens = preprocess([query], True, True)[0]
scores = cosine_similarity(query_tokens, champion_list, inverted_index, N, docs_norms)
print_result(json_data, scores, k=5)

title:  نکونام: نفتی ها بهترین بازی خود را انجام دادند/بازیکنان جدیدمان کیفیت بالای خود را نشان دادند 
url:  https://www.farsnews.ir/news/14001204001165/نکونام-نفتی-ها-بهترین-بازی-خود-را-انجام-دادند-بازیکنان-جدیدمان-کیفیت
title:  ماجدی: فوتبال کشور به تغییرات نیاز دارد 
url:  https://www.farsnews.ir/news/14001223000539/ماجدی-فوتبال-کشور-به-تغییرات-نیاز-دارد
title:  تقدیر مربی تیم فوتبال خلیج فارس از ماجدی بابت رسیدگی به شائبه تبانی در لیگ جوانان 
url:  https://www.farsnews.ir/news/14001222000297/تقدیر-مربی-تیم-فوتبال-خلیج-فارس-از-ماجدی-بابت-رسیدگی-به-شائبه-تبانی-در
title:  دیدار مدیر تیم ملی ایران با سفیر کره جنوبی/ هیان: امیدوارم مقابل ایران مساوی کنیم 
url:  https://www.farsnews.ir/news/14001213000138/دیدار-مدیر-تیم-ملی-ایران-با-سفیر-کره-جنوبی-هیان-امیدوارم-مقابل-ایران
title:  امیدواری ملی پوش سابق فوتبال ساحلی بابت تغییرات در کادرفنی تیم ملی 
url:  https://www.farsnews.ir/news/14000927000275/امیدواری-ملی-پوش-سابق-فوتبال-ساحلی-بابت-تغییرات-در-کادرفنی-تیم-ملی


In [309]:
query = 'تیم ملی فوتبال ایران'
query_tokens = preprocess([query], True, True)[0]
scores = cosine_similarity(query_tokens, champion_list, inverted_index, N, docs_norms)
print_result(json_data, scores, k=5)

title:  اسکوچیچ: مردم متوجه شده‌اند که می‌توانند هدایت تیم ملی را به من واگذار کنند/هاشمیان سواد اروپایی از فوتبال دارد 
url:  https://www.farsnews.ir/news/14001222000329/اسکوچیچ-مردم-متوجه-شده‌اند-که-می‌توانند-هدایت-تیم-ملی-را-به-من-واگذار
title:  مصاحبه فارس با کارشناس فوتبال آسیا| از میراث بزرگ کی‌روش و قدرت ایران با اسکوچیچ تا انقلاب برانکو در عمان 
url:  https://www.farsnews.ir/news/14001124000522/مصاحبه-فارس-با-کارشناس-فوتبال-آسیا|-از-میراث-بزرگ-کی‌روش-و-قدرت-ایران
title:  مصاحبه فارس با یک مسؤول فدراسیون عراق: «دربی آسیا» جذاب و غیرقابل پیش‌بینی است/ ایران زیرنظر اسکوچیچ تاکتیک‌پذیرتر از دوران کی‌روش 
url:  https://www.farsnews.ir/news/14001104000095/مصاحبه-فارس-با-یک-مسؤول-فدراسیون-عراق-دربی-آسیا-جذاب-و-غیرقابل
title:  مصاحبه فارس با خبرنگار عراقی| از پیشرفت ایران با اسکوچیچ، طارمی و آزمون تا تاثیر مکتب حاج قاسم در جهان 
url:  https://www.farsnews.ir/news/14001014000335/مصاحبه-فارس-با-خبرنگار-عراقی|-از-پیشرفت-ایران-با-اسکوچیچ-طارمی-و-آزمون
title:  نکونام: نفتی ها بهترین بازی خو

In [274]:
query = 'واترپلو'
query_tokens = preprocess([query], True, True)[0]
scores = cosine_similarity(query_tokens, champion_list, inverted_index, N, docs_norms)
print_result(json_data, scores, k=5)

title:  تجلیل از خانواده شهید حسن نوفلاح با تقدیم مدال قهرمانی 
url:  https://www.farsnews.ir/news/14001205000938/تجلیل-از-خانواده-شهید-حسن-نوفلاح-با-تقدیم-مدال-قهرمانی
title:  سرپرست  فدراسیون شنا، شیرجه و واترپلو  منصوب شد 
url:  https://www.farsnews.ir/news/14001011000202/سرپرست-فدراسیون-شنا-شیرجه-و-واترپلو-منصوب-شد
title:  رضوانی رئیس فدراسیون شنا ماند/ 3 رای سفید برای تنها کاندیدا 
url:  https://www.farsnews.ir/news/14001020000253/رضوانی-رئیس-فدراسیون-شنا-ماند-3-رای-سفید-برای-تنها-کاندیدا
title:  گرفتن ۷۰ نمونه تست دوپینگ در ۸ رشته طی یک هفته 
url:  https://www.farsnews.ir/news/14001207000351/گرفتن-۷۰-نمونه-تست-دوپینگ-در-۸-رشته-طی-یک-هفته
title:  برنامه‌های رئیس فدراسیون شنا برای 4 سال آینده/ رضوانی: تلاش می‌کنیم به اهداف‌مان در بازی‌های آسیایی برسیم 
url:  https://www.farsnews.ir/news/14001020000394/برنامه‌های-رئیس-فدراسیون-شنا-برای-4-سال-آینده-رضوانی-تلاش-می‌کنیم-به


In [315]:
query = 'واکسن کرونا ایرانی'
query_tokens = preprocess([query], True, True)[0]
scores = cosine_similarity(query_tokens, champion_list, inverted_index, N, docs_norms)
print_result(json_data, scores, k=5)

title:  مرندی: رهبر انقلاب دُز سوم واکسن کرونا را دریافت کرده‌اند 
url:  https://www.farsnews.ir/news/14001117000930/مرندی-رهبر-انقلاب-دُز-سوم-واکسن-کرونا-را-دریافت-کرده‌اند
title:  الهیان: وزارت بهداشت درباره علت عدم پیش خرید واکسن فخرا توضیح دهد 
url:  https://www.farsnews.ir/news/14000729000462/الهیان-وزارت-بهداشت-درباره-علت-عدم-پیش-خرید-واکسن-فخرا-توضیح-دهد
title:  رئیسی: مسافرت با رعایت اصول بهداشتی بلااشکال است 
url:  https://www.farsnews.ir/news/14001214000475/رئیسی-مسافرت-با-رعایت-اصول-بهداشتی-بلااشکال-است
title:  آیت‌الله رئیسی: اقناع افکار عمومی در مسیر مقابله با کرونا ضروری است/ وزارت بهداشت از واکسن‌های تولید داخل حمایت کند 
url:  https://www.farsnews.ir/news/14000801000317/آیت‌الله-رئیسی-اقناع-افکار-عمومی-در-مسیر-مقابله-با-کرونا-ضروری-است-
title:  نایب رئیس مجلس: دولت در تزریق واکسن و کاهش نگرانی‌‌های مردم شاهکار کرده است 
url:  https://www.farsnews.ir/news/14000923000300/نایب-رئیس-مجلس-دولت-در-تزریق-واکسن-و-کاهش-نگرانی‌‌های-مردم-شاهکار
